In [2]:
## Install from command prompt. Jupyter installation is loading the packages to wrong library

# %pip install --upgrade pip
# %pip install --disable-pip-version-check \

#%pip install transformers==4.27.2

In [90]:
#!pip show transformers

In [4]:
#pip install torch==1.13.1

### Using Open source flan model

In [1]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [2]:
model_name='google/flan-t5-xl'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

#### Zero-shot inference

In [15]:
zero_shot_prompt = f"""

The Price of SKU is 15899

Offers:
1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999
4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

what is the maximum discount? 
"""

In [16]:
print(zero_shot_prompt)



The Price of SKU is 15899

Offers:
1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999
4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

what is the maximum discount? 



In [17]:
inputs = tokenizer(zero_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=500,
    )[0], 
    skip_special_tokens=True
)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

MODEL GENERATION - ZERO SHOT:
750


#### One-shot inference

In [7]:
prompt = f"""

Q. 

The Price of SKU is 17999

Offers:

1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999
5. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

A: The price of the SKU is 17999. From the first offer the maximum dicount is 5% of price if SKU which is 17999) = 900. From the second offer the maximum discount is 100+500=600. From the third offer discount = 500 since the purchase value 17999 is greater than Minimum purchase value of 15999. From the fourth offer the discount = 750 since the Minimum pucrhase value of 15999 is less than than sku price of 17999. Out of the 4 offers the maximum discount is 900. So the answer is 900.


Q.

The Price of SKU is 15899

Offers:
1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999
4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

A: 
"""

In [8]:
print(prompt)



Q. 

The Price of SKU is 17999

Offers:

1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999
5. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

A: The price of the SKU is 17999. From the first offer the maximum dicount is 5% of price if SKU which is 17999) = 900. From the second offer the maximum discount is 100+500=600. From the third offer discount = 500 since the purchase value 17999 is greater than Minimum purchase value of 15999. From the fourth offer the discount = 750 since the Minimum pucrhase value of 15999 is less than than sku price of 17999. Out of the 4 

In [9]:
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=500,
    )[0], 
    skip_special_tokens=True
)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

MODEL GENERATION - ONE SHOT:
The price of the SKU is 15899. From the first offer the maximum dicount is 5% of price if SKU which is 15899) = 900. From the second offer the maximum discount is 100+500 = 600. From the third offer the discount = 500 since the purchase value 15999 is less than than sku price of 15899. From the fourth offer the discount = 750 since the minimum purchase value is 15999. So the answer is 750.


#### few shots inference

In [10]:
prompt = f"""

Q. 

The Price of SKU is 17999

Offers:

1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999
5. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

A: The price of the SKU is 17999. From the first offer the maximum dicount is 5% of price if SKU which is 17999) = 900. From the second offer the maximum discount is 100+500=600. From the third offer discount = 500 since the purchase value 17999 is greater than Minimum purchase value of 15999. From the fourth offer the discount = 750 since the Minimum pucrhase value of 15999 is less than than sku price of 17999. Out of the 4 offers the maximum discount is 900. So the answer is 900.


Q.

The Price of SKU is 15899

Offers:
1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999
4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

A: The price of the SKU is 15899. From the first offer the maximum dicount is 5% of price if SKU which is 15899) = 795. From the second offer the maximum discount is 100+500=600. From the third offer since the purchase value 15899 is less than minimum sku purchase value of 15999, hence the discount is not applicable. From the fourth offer since the minimum pucrhase value of 15999 is greater than the sku price of 15899, hence the discount is not applicable. Out of the 4 offers the maximum discount is 795. So the answer is 795.
"""

### using openai - gpt-3.5-turbo

#### zero-shot inference

In [18]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "The Price of SKU is 17999\n\nOffers:\n\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n\nWhat is the maximum discount applicable?"}
]

In [19]:
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)

<OpenAIObject chat.completion id=chatcmpl-7iHv5NFxP9Z07xZkoOKbVub7J29uV at 0x262771bbab0> JSON: {
  "id": "chatcmpl-7iHv5NFxP9Z07xZkoOKbVub7J29uV",
  "object": "chat.completion",
  "created": 1690789671,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The maximum discount applicable is INR 750, which is offered through the \"Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn\" offer."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 172,
    "completion_tokens": 36,
    "total_tokens": 208
  }
}

#### one-shot inference

In [13]:
messages = [
    {"role": "system", "content": "You are a helpful assistant which helps in the calculation of the maximum discount. End your answer with maximum discount amount"},
    {"role": "user", "content": "The Price of SKU is 17999\n\nOffers:\n\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n\nWhat is the maximum discount applicable?"},
    {"role": "assistant", "content": "Assuming prime member from the first offer the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of price of sku, which is 17999. This equals 900. From the second offer the maximum discount is 100+500=600. From the third offer since the purchase value 17999 is greater than Minimum purchase value of 15999, hence the discount amount is applicable & it is equal to 500. From the fourth offer the discount = 750 since the Minimum pucrhase value of 15999 is less than than 17999. Out of the 4 offers the maximum discount is 900. So the answer is 900"},
    {"role": "user", "content": "The Price of SKU is 15899\n\nOffers:\n\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n\nwhat is the maximum discount applicable?"}
]


In [14]:
import openai

output=openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)

print(f"ONE SHOT INTERFERENCE-OPEN AI: \n{output}")

ONE SHOT INTERFERENCE-OPEN AI: 
{
  "id": "chatcmpl-7iFkXzVLirzOwhn8PanuZplfzYUFy",
  "object": "chat.completion",
  "created": 1690781329,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Assuming the customer is a Prime member from the first offer, the maximum discount applicable is 5% since it is greater than 3%. So, the discount amount is 5% of the price of the SKU, which is 15899. This equals 794. From the second offer, the maximum discount is 100 + 500 = 600. From the third offer, since the purchase value of 15899 is less than the minimum purchase value of 15999, the discount is not applicable. From the fourth offer, the discount is also not applicable since the minimum purchase value of 15999 is not met. Out of the four offers, the maximum discount is 794. So, the maximum discount applicable is 794."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 497,
 

#### few-shots inference

In [2]:
messages_2 = [
    {"role": "system", "content": "You are a helpful assistant which helps in the calculation of the maximum discount. If any offer have more than one discount % take the higher one. End your answer with maximum discount amount"},
    {"role": "user", "content": "The Price of SKU is 17999\n\nOffers:\n\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n\nWhat is the maximum discount applicable?"},
    {"role": "assistant", "content": "Assuming prime member from the first offer the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of price of sku, which is 17999. This equals 900. From the second offer the maximum discount is 100 cashback + 500 welcome rewards= 600. so the discount from this offer is 600. From the third offer since the purchase value 17999 is greater than Minimum purchase value of ₹15999, hence the discount amount is applicable & it is equal to ₹500. From the fourth offer the discount = ₹750 since the Minimum pucrhase value of ₹15999 is less than than ₹17999. Comparing the discounts from the 4 offers the maximum discount is ₹900. So the answer is ₹900"},
    {"role": "user", "content": "The Price of the SKU is 18999\n\nOffers:\n\n1. Flat INR 2000 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n\n6. ₹100 cashback & ₹2150 welcome rewards on Amazon Pay Later.\n\nwhat is the maximum discount applicable?"},
    {"role": "assistant", "content": "From the first offer since the price value of 18999 is greater than 17999 the discount amount is 2000 . From the second offer since the price value of 18999 is greater than 17999 the discount amount is 2100. From the third offer since the price value of 18999 is less than minimum purchase value of 19999, the discount value is not applicable. From the fourth offer, the discount amount is 5% of the price of the SKU, up to a maximum of 250. Since the price of the SKU is 18999, the discount amount will be 250. Assuming prime member, from the fifth offer, the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of the price of the SKU, which is 18999. This equals 950.From the sixth offer, the discount amount is 100 cashback + 2150 welcome rewards = 2250. Comparing the discounts from the six offers, the maximum discount is 2250. So,the maximum discount applicable is 2250."},
    {"role": "user", "content": "The Price of SKU is 15899\n\nOffers:\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n\nwhat is the maximum discount?"}
]


In [3]:
import openai

output=openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages_2
)

print(f"FEW SHOT INTERFERENCE-OPEN AI: \n{output['choices'][0]['message']['content']}")

FEW SHOT INTERFERENCE-OPEN AI: 
From the first offer assuming prime member, the discount amount is 5% of the price of the SKU, which is 15899. This equals 794. From the second offer, the discount amount is 100 cashback + 500 welcome rewards= 600. So the discount from this offer is 600. From the third offer, since the purchase value 15899 is less than the minimum purchase value of ₹15999, the discount amount is not applicable. From the fourth offer, the discount is ₹750 since the minimum purchase value of ₹15999 is greater than the price of SKU. Comparing the discounts from the four offers, the maximum discount is ₹794. So, the maximum discount applicable is ₹794.
